## Numerical Exercises for E&M :  Charges, Method of Images, Boundary Value Problems


Make a copy and return the notebook, but only include the code and plots for the exercises. Include plot titles on all plots. Discuss the results for each exercise. Additional exploration is encouraged!

See  http://matplotlib.org/examples/color/colormaps_reference.html for color tables.

## 1.  Equipotentials and Electric Field Lines: point charges

We'll start with taking a look at the electric field and potential for a system of point charges. Recall that equipotentials are the family of curves V =  constant and $\vec{E} = - \nabla V$.

### Electric Field
$$ \vec{E}(\vec{r}) = \frac{1}{4\pi\epsilon_0}\sum_i \frac{q_i\,\left(\vec{r}-\vec{r}_i\right)}{\left\vert \vec{r} - \vec{r}_i \right\vert^3} $$

### Electric Potential

$$ V(\vec{r}) = \frac{1}{4\pi\epsilon_0} \sum_i \frac{q_i}{\left\vert \vec{r} - \vec{r}_i \right\vert} $$

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate
from __future__ import division
import matplotlib.cm as cm
import check             # see check.py  just checks array shape, max, min

k = (9E9) * (1E-9)    # Will yield E in units of Volts/meter, and V in Volts.
#set some default plot parameters
#plt.rcParams.keys()     # This is a dictionary that lists all plot parameters

plt.rc("xtick", labelsize="large")
plt.rc("ytick", labelsize="large")
plt.rc("axes", labelsize="xx-large")
plt.rc("axes", titlesize="xx-large")
sz = 8.
plt.rc("figure", figsize=(1.25*sz,sz))  # gives a figure with equal aspect ratio;
# or can also do by adding to the figure
#xx = plt.axes().set_aspect('equal')

The functions defined below implement the calculation and plotting of both equipotential lines (via a contour plot) and electric field lines (via the stream plot function). 

## 1.1 Functions to create $\vec{E}$, V, and plots.

The *V* function for potential and the *Efield* function for electric field take as argument the $x$ and $y$ coordinates of the field point, and then the list of electric charges. Here, we're looking at V and $\vec{E}$ in the plane z=0.  

The list of electric charges, *qr*, is set up so that each charge *q* in *qr* is a three-element tuple. Put the charges at z=0, and plot the fields in the plane z=0. For each *q* in *qr*:

  * q[0] is the charge's $x$ coordinate.
  * q[1] is the charge's $y$ coordinate.
  * q[2] is the electric charge. 
  
The Coulomb constant $k=\frac{1}{4\pi \epsilon_0}$ is scaled in a way that assumes the charges will be given in units of nC.

The code also takes care to avoid having the field point at the location of the point charge.

In [ ]:
# One point charge at (x, y, 0)
qr = [(0.0,0.0,1.0)]  # x, y, Q. z = 0 . Assume the given charges are in nC
k = (9E9) * (1E-9)    # Will yield E in units of Volts/meter, and V in Volts.
radius = 0.02         # "dead zone" to prevent calculating E or V on top of a charge
x = np.arange(-10.0,10.0,0.1)
y = np.arange(-10.0,10.0,0.1)
X,Y = np.meshgrid(x,y)
def V(x, y, qr):
    Vr = 0.0
    for q in qr:
        r = np.sqrt((x - q[0])**2 + (y - q[1])**2)
        if (r >= radius):
            Vr += k*q[2] / r
    return Vr

def Efield(x, y, qr):
    Ex = 0.0
    Ey = 0.0
    for q in qr:
        if ((x-q[0])**2 + (y-q[1])**2 > radius**2):   # a safety "dead zone"
            
            Ex += k*q[2]*(x-q[0])/((x-q[0])**2 + (y-q[1])**2)**1.50
            Ey += k*q[2]*(y-q[1])/((x-q[0])**2 + (y-q[1])**2)**1.50

    return Ex, Ey

The next function defines the plotting function.  It takes as argument the list of point charges *qr*.  

It plots 

  * A contour plot for the electric potential.
  * An extra thick contour line at V=0, for highlighting purposes.
  * Optional white contour lines (sometimes useful).
  * A stream plot for the electric field. 
 
  
Note how Varr and Earr are defined in the code below - a cool way to loop over all the charges in qr.  

In [ ]:
def Plots(qr, title = ""):
    colormap = cm.get_cmap('rainbow')
    Varr = np.array([[V(ix,iy,qr) for ix in x] for iy in y ])  
    #print(np.min(Varr), np.max(Varr))
    Earr = np.array([[Efield(ix,iy,qr) for ix in x] for iy in y])
    print(np.shape(Earr[:,:,0]), np.shape(Earr[:,:,1]))
    # specify which contours to plot:
    equipot = plt.contourf(X,Y,Varr, levels=np.linspace(0.5*np.min(Varr), 0.5*np.max(Varr), 40), cmap=colormap)
    # or have contour levels chosen automatically:
    #equipot = plt.contour(X,Y,Varr, 40)   
    plt.contour(X,Y,Varr, levels=[0.0], linewidths=3.0)  #overplot V = 0 contour
    #overplot line contours
    plt.contour(X,Y,Varr, levels=np.linspace(0.5*np.min(Varr), 0.5*np.max(Varr), 40), colors='white')

    plt.streamplot(x,y, Earr[:,:,0], Earr[:,:,1], color="black", density=3)
    plt.colorbar(equipot)
    plt.xlim(-7.0, 7.0)
    plt.ylim(-7.0, 7.0)
    plt.xlabel("$x$")
    plt.ylabel("$y$")
    plt.title(title)

Next,

  1. Set up the list of charges *qr*.
  2. Call the plotting function, which will plot *both* the electric field lines and the equipotential lines.
  

### Example: Single point charge

Note that for a single point charge, V = 0 only at $\infty$.

In [ ]:
qr = []
qr.append((0.0, 0.0, 1.0))

In [ ]:
Plots(qr, title = "Electric Field Lines and Equipotentials\n for Single Point Charge")

#### Discussion
The electric field lines "radiate" out away from the positive charge, as we would expect.  The equipotentials form spheres around the charge.  At every point, the electric field lines and the equipotentials are perpendicular to each other. 


### Example: Two positive charges

In [ ]:
qr = []
qr.append((1.0, 0.0, 1.0))
qr.append((-1.0, 0.0, 1.0))
Plots(qr, title = "Electric Field Lines and Equipotentials\n for Two Positive Charges in the plane z = 0")

#### Discussion
With the two like charges, we see the electric field lines "steer away" from the region in between the charges.  The equipotential surfaces also form an interesting saddle region between the charges.  Far away from the charges, the electric field lines and equipotential surfaces begin to look like those of a single point charge with charge Q = 2q.  


<div class="alert alert-block alert-warning">
Exercise 1: Simulate a line charge by placing 5 positive point charges (q = 1) between x = -2 and x = 2, y = 0. About how far from the origin does the potential look like the potential of a point charge with Q = 5q? (Overplot the potential for Q = 5q for a field point far from the line, and overplot line contours for the potential.) You may have to increase the range of x and y. Hint: try qr = [(-2,0,1),(-1,0,1),(0,0,1),(1,0,1),(2,0,1)] and then run Plots as in the previous example.

In [ ]:
qr = [(-2,0,1),(-1,0,1),(0,0,1),(1,0,1),(2,0,1)]         #Exercise 1 solution 
Plots(qr, title = "Line of charges")

**Discussion 1:** Type your discussion here.

###  Example: A positive and negative charge (dipole)

In [ ]:
qr = []
qr.append((2.0, 0.0, 1.0))
qr.append((-2.0, 0.0, -1.0))

In [ ]:
Plots(qr, title = "Electric Field Lines and Equipotentials\n for Opposite Charges")

**Discussion:**

A major feature that we see is that the electric field lines point from the positive charge and towards the negative charge. As always, the electric field lines are perpendicular to the equipotential lines. 

### 2. Method of Images for Boundary Value Problems

#### Example:  Point charge near a conducting plane
  
Find the potential and electric field for a point charge near a grounded, conducting plane. V = 0 on the plane (E, V = 0 inside the conductor).
 
With the $V=0$ equipotential plane at $x=0$, the dipole reproduces the boundary conditions for the problem of a point charge near a conducting plane with V=0 on the plane. Since the solution is unique, this is the solution to the boundary value problem in the region $x>0$. This is the so-called "method of images".  

The surface charge density induced on the conducting plate can be found from the boundary conditions on the normal component of $\vec{E}$:

$$ \sigma(y, z) = \epsilon_0 \left(\vec{E}_\mathrm{outside} - \vec{E}_\mathrm{inside}\right)\cdot \hat{n}, $$

where in this case $\hat{n} = \hat{x}$ and $\vec{E}_\mathrm{inside} = 0$.  Then, we are plotting the $x$-component of the electric field for all $y$ at $x=z=0$.  

In the plot below of $\sigma(y, z=0)$, it makes sense that most of the negative charge induced on the plate is concentrated at $y=0$, which is closest to the positive charge. 

In [ ]:
# Plot of the surface charge density, for the method of images solution. 
sigma = [Efield(0.0, iy, qr)[0]/(4.0*np.pi*k) for iy in y]
plt.plot(y, sigma)
plt.xlabel("$y$")
plt.ylabel("$\sigma(y)$")
plt.title("Induced Surface Charge Density vs. Vertical Position of the point charge")

#### Example: Point charge $q_a$ near a conducting sphere, radius R.  BC: V = 0 at r = R.
Note that E = 0 inside the sphere.


In [ ]:
qr = []
a = 4.0
qa = 1.0
R = 2.0
qb = -qa*R/a
b = (R**2)/a
print(qb, b)
qr.append((a, 0.0, qa))
qr.append((b, 0.0, qb))
Plots(qr, title = "Electric Field Lines and Equipotentials\n for a point charge near a conducting sphere")

#### Discussion
 
The $V=0$ equipotential surface forms a sphere, so this pair of opposite charges reproduces the boundary conditions with $q_b = -q_a\frac{R}{a}$ at $b =\frac{R^2}{a}$.  The solution to the method of images problem can be viewed by plotting this figure, keeping in mind that it is valid only outside the sphere. We know that V=0 inside the sphere.  

<div class="alert alert-block alert-warning">
Exercise 2: For the given parameters, qb = -0.5. If we take qb = -qa = -1, we have again the dipole with the V = 0 surface the y-z plane. Keeping a, b and R the same, change the value of qb (consider $-1 < qb < -0.5$). Is the equipotential surface V = 0 still a sphere? What happens when $qb < -1$? Hint: try qr = [(1,0,-0.75),(4,0,1)], then run Plots.

In [ ]:
qr = [(1,0,-0.75),(4,0,1)]
Plots(qr, title = "Image charge")                                            #solution to Exercise 2

**Discussion 2:** Type your discussion here.

#### Example:  Four point charges

In [ ]:
qr = []
qr.append((1.0, 1.0, 1.0))
qr.append((-1.0, 1.0, -1.0))
qr.append((-1.0, -1.0, 1.0))
qr.append((1.0, -1.0, -1.0))
Plots(qr, title = "Electric Field Lines and Equipotentials\n for Four Charges")

#### Discussion
We see the electric field lines of a *quadrupole* charge distribution.  As before, we see the field lines emanate from the positive charges and "go into" the negative charges.  We see the $V=0$ equipotential surface forming two planes, both $x=0$ and $y=0$. This distribution of image charges is the solution to the method of images problem for say a positive point charge at (1,1,0) near a conducting plane bent into a $90^\circ$ angle. The solution is valid in the region $x>0, y>0$.  


<div class="alert alert-block alert-warning">
Exercise 3: What collection of charges would solve the image problem for a conducting plate bent into a $60^\circ$ angle? Plot V and the E field lines. *Hint: All you have to do is to create a different qr array. Consider placing charges at the locations (x,y) = (cos 2$\pi$n/Nq, sin 2$\pi$n/Nq) where Nq = number of charges and n=0,...Nq-1.*

In [ ]:
#                                                # Solution to Exercise 3         
qr = []
Nq = 6
for n in range(0, Nq):
    xn = np.cos(2.0*np.pi * n / Nq)
    yn = np.sin(2.0*np.pi * n / Nq)
    qn = (-1)**n
    qr.append((xn, yn, qn))
Plots(qr, title = "Electric Field Lines and Equipotentials\n for Six Charges")

**Discussion 3:** Type your discussion here.

#### Line charge  near a conducting plane

<div class="alert alert-block alert-warning">
Exercise 4: The potential for an $\infty$ thin straight wire at $(x_o, y_o)$ running from $- \infty < z < \infty$ is V(s)= - 2k$\lambda$ ln s where s is the distance from the wire $s = [(x-x_o)^2 + (y-y_o)^2]^{1/2}$ and $\lambda$ is the charge per unit length. The electric field is

$$\vec{E} = (\frac{2k\lambda}{s^2})[(x-x_o)\hat{x}+ (y-y_o)\hat{y}] $$.

The cells below are scripts to 1) compute V and E for any number of line charges going into the page and 2) plot V and ${\vec E}$ .  For a negative line charge at (-2,0) and a positive line charge at (2,0), create a cell with the lines $\\$
    
    qr = [(-2,0,-1), (2,0,1)]
    Plots_line(qr, title="Line charge dipole") $\\$
    
What is the shape of the equipotentials compared to those for the point dipole? Overplot line contours for the potential so you can see them better. Take $\lambda = 1$.

### Compute the potential and E for any number of line charges.

In [ ]:
def V_line(x, y, qr):                                                              # Exercise 4 solution
    Vr = 0.0
    for q in qr:
        r = np.sqrt((x - q[0])**2 + (y - q[1])**2)
        if (r >= radius):
            Vr += 2*k*q[2]*np.log(r)
    return Vr

def Efield_line(x, y, qr):
    Ex = 0.0
    Ey = 0.0
    for q in qr:
        if ((x-q[0])**2 + (y-q[1])**2 > radius**2):   # a safety "dead zone"
            
            Ex += 2*k*q[2]*(x-q[0])/((x-q[0])**2 + (y-q[1])**2)
            Ey += 2*k*q[2]*(y-q[1])/((x-q[0])**2 + (y-q[1])**2)

    return Ex, Ey

### Create a function to plot the potential and E for any number of line charges.

In [ ]:
def Plots_line(qr, title = "", xlim=[-8.0,8.0], ylim=[-8.0,8.0]):                 # Exercise 4 solution
    colormap = cm.get_cmap('rainbow')
    Varr = np.array([[V_line(ix,iy,qr) for ix in x] for iy in y ])  
    #print(np.min(Varr), np.max(Varr))
    Earr = np.array([[Efield_line(ix,iy,qr) for ix in x] for iy in y])
    print(np.shape(Earr[:,:,0]), np.shape(Earr[:,:,1]))
    # specify which contours to plot:
    equipot = plt.contourf(X,Y,Varr, levels=np.linspace(0.5*np.min(Varr), 0.5*np.max(Varr), 40), cmap=colormap)
    # or have contour levels chosen automatically:
    #equipot = plt.contour(X,Y,Varr, 40)   
    plt.contour(X,Y,Varr, levels=[0.0], linewidths=3.0)  #overplot V = 0 contour
    #overplot line contours
    plt.contour(X,Y,Varr, levels=np.linspace(0.5*np.min(Varr), 0.5*np.max(Varr), 40), colors='brown')

    plt.streamplot(x,y, Earr[:,:,0], Earr[:,:,1], color="black", density=3)
    plt.colorbar(equipot)
    plt.xlim(xlim)
    plt.ylim(ylim)
    plt.xlabel("$x$")
    plt.ylabel("$y$")
    plt.title(title)

In [ ]:
qr = [(-2,0,-1), (2,0,1)]
Plots_line(qr, title="Line charge dipole")                                          # Exercise 4 solution

**Discussion 4:** Type your discussion here.

# 3.  Boundary value problems for Laplace's Equation

In a charge-free region, we seek solutions to Laplace's equation, 
$$ \nabla^2 V = 0. $$ with prescribed boundary conditions.

## 3.1 Cartesian coordinates, no z dependence. 

We solve Laplace's equation using separation of variables: V(x,y) = X(x)Y(y). The domain is $ 0 \le y \le a$ and $0 \le x < \infty$.

First consider the problem V = 0 at y = 0 and y = a, with V(x =0, y) = $V_o(y)$. The general solution that satisfies the BC at y = 0 and y = a and that vanishes for $x\rightarrow \infty$ is
$$ V(x,y) = \sum_{n=1}^\infty A_n e^{-n\pi x/a} \sin\frac{n\pi y}{a}, $$
with 
$$V_o(y) = \sum_{n=1}^\infty A_n \sin\frac{n\pi y}{a} $$
and the coefficients $A_n$ are given by
$$ A_n = \frac{2}{a} \int_0^a V_0(y) \sin\frac{n\pi y}{a}\,\mathrm{d}y. $$ 
Recall that the functions $\sin\frac{n\pi y}{a}$ are a basis for all square-integrable functions f(y) on [0,a], where f(0) = f(a) = 0, i.e. any f(y) can be expanded in this basis. The basis functions are orthogonal,

$$\int_0^a \sin\frac{n\pi y}{a} \sin\frac{m\pi y}{a}\,\mathrm{d}y = \frac{a}{2}\delta_{nm}. $$

The integral for the $A_n$ will be computed by using quad, a general purpose scipy integrator that uses Gaussian quadrature (https://en.wikipedia.org/wiki/Gaussian_quadrature).

In [ ]:
# Given a boundary condition V0(y), compute the A_n and plot V(x,y)

# constants                                       
a = 1.0

# a "dummy" definition of V_0, to be redefined later
def V0(y):
    return 1.0
# integration kernel for the A_n coefficients
def AnKernel(y, n):
    return V0(y) * np.sin(n*np.pi * y / a)

# function to precompute the A_n values
#N = 20    # Maximum number of terms to keep
def AnCompute(N):
    A = np.zeros(N)
    for n in range(1, N+1):
        A[n-1] = 2.0*scipy.integrate.quad(AnKernel, 0.0, a, args=(n))[0]/a
    return A
def VCompute(x, y, A):
    N = len(A)
    Vr = 0.0  # return value
    for n in range(1, N+1):
        Vr += A[n-1] * np.exp(-n*np.pi*x / a) * np.sin(n*np.pi *y / a)
    return Vr

def RunAndPlot(A):
    colormap = cm.get_cmap('jet')
    yarr = np.linspace(0.0, a, 100)
    xarr = np.linspace(0.0, 5*a, 100)
    X, Y = np.meshgrid(xarr, yarr)
    V = VCompute(X, Y, A)
    equipot = plt.contourf(X, Y, V, 40, cmap = colormap)
    plt.contour(X, Y, V, 40,colors="black")

    plt.colorbar(equipot)
    plt.xlabel("$x$")
    plt.ylabel("$y$")
    

***Example:*** $ V_0(y) = \sin\left(\frac{2\pi y}{a}\right): $

In [ ]:
# define the V_0 function
def V0(y):
    return 2.*np.sin(2.0*np.pi * y / a)
# Compute A_n
N = 10
A = AnCompute(N)
print("A = ")
print(A)

**Note:** The above shows that the only non-zero coefficient is $A_2$ = 2.0 because $V_o(y)$ projects only on to the n=2 basis function.

In [ ]:
# Compute V and plot
RunAndPlot(A)
plt.xlim(0.0, 0.6)
plt.title(r"$V(x,y)$ for $V_0(y) = 2.0\sin\left( \frac{2\pi y}{a} \right)$")

 <div class="alert alert-block alert-warning">
Exercise 5:
Compute and plot V(x,y) as above for a multi-scale boundary condition.   Consider the BC: $\\$
    $V_o(y)=3sin(\pi y/a)+ 3sin(3\pi y/a)+6sin(5\pi y/a). \\$  Discuss the result, especially how large and small scales penetrate into the region with increasing x.$\\$
    

In [ ]:
# define the V_0 function                                                                      #Exercise 5 solution
def V0(y):
    return 3.0*np.sin(3.0*np.pi * y/a) + 3.0*np.sin(np.pi*y/a) + 6.*np.sin(5.*np.pi*y/a)

N = 10
A = AnCompute(N)
print(A)

RunAndPlot(A)
plt.xlim(0,0.8)
plt.title(r"$V(x,y)$")

**Discussion 5:** Type your discussion here.

***Example:*** $ V_0(y) = -y^2 + ay$, a=1.0.

In [ ]:
# define the V_0 function
def V0(y):
    return (-y**2 + a*y)
# Precompute the A_n
N = 100  # number of terms in the sum
A = AnCompute(N)
# display...
print(A[0:10])
# Compute V and plot
RunAndPlot(A)
plt.xlim(0.0, 1.5)
plt.title("$V(x,y)$ for $V_0(y) = -y^2 + ay$")

In the plot above, all 100 computed terms were included.  In the plot below, $A_n$ will be plotted vs. $n$, to show that only the first few terms $A_n$ contribute to the sum.  

In [ ]:
plt.plot(range(1, N+1), A)
plt.xlim(0,10)
plt.title("$A_n$ vs. $n$, $V_0(y) = -y^2 + ay$")
plt.xlabel("$n$")
plt.ylabel("$A_n$")

<div class="alert alert-block alert-warning">
Exercise 6: Take $ V_0(y) = - 5(y-0.5)^2 + y(y-a)^3 - 10(y-a)y^4. $ Plot $ V_0(y)$, and compute and plot the solution and E field lines. Plot the coefficients $A_n$ as above and comment on the number of terms required compared to the case above. 

In [ ]:
V0 = lambda y : -5*(y-0.5)**2 + y*(y-a)**3 - 10*(y-a)*y**4            #Exercise 6 solution
t = np.linspace(0,1,100)
plt.plot(t,V0(t))
plt.plot(t,np.zeros_like(t),color='black')
plt.title('Boundary potential $V_0(y)$')
plt.ylabel('$V_0$')
plt.xlabel('$y$')
plt.grid()
plt.show()

In [ ]:
N = 100                                                                 #Exercise 6 solution
A = AnCompute(N)
#print(A)

RunAndPlot(A)
plt.xlim(0,1)
plt.title(r"$V(x,y)$")

plt.figure()
plt.bar(np.arange(N)+1, A)
#plt.plot(np.arange(N)+1, np.zeros(N), color='black')
plt.title('Coefficients')
plt.xlim(0,100)
plt.xlabel('n')
plt.ylabel('$A_n$')

plt.show()

**Discussion 6:** Type your discussion here.

### 3.2 Separation of Variables - Spherical Coordinates

In spherical coordinates, *with azimuthal symmetry*, Laplace's equation has series solutions of the form

$$ V(r, \theta) = \sum_{\ell=0}^{\infty} \! \left( A_\ell r^\ell + \frac{B_\ell}{r^{\ell+1}} \right) P_\ell (\cos\theta) $$

where $P_{\ell} (cos\theta)$ are the Legendre polynomials.

#### Graph the first few Legendre polynomials, $\ell=0$ to $\ell=5$ ($x \equiv cos \theta$).

In [ ]:
import scipy.special as sf     #special functions

xarr = np.linspace(-1.0,1.0,100)    #x = cos(theta)
#Pn = []
for n in range(0, 5+1):
    Pn = (sf.eval_legendre(n, xarr))   #get the Legendre polynomials
    plt.plot(xarr, Pn, label=r"$n=${:}".format(n))

plt.xlabel("$x$")
plt.ylabel("$P_n(x)$")
plt.title(r"Legendre Polynomials, from $n=0$ to $n=5$")
plt.legend(fontsize="xx-large")
plt.ylim(-1.0,1.1)

The first two Legendre polynomials are: (x = cos $\theta$)

$$P_o(x) = 1$$
$$P_1(x) = x$$

and $P_\ell$ for $\ell \ge 2$ can be generated from the recursion relation:

$$(\ell + 1)P_{\ell+1}(x) = (2\ell + 1)x P_\ell (x) - \ell P_{\ell-1}(x).$$

### 3.3 A Sphere with a prescribed  potential on the surface.
The potential on the surface of a sphere of radius $R$ is given by a function $V_0(\theta)$.  

We know a few conditions: 

 - $r<R$: $B_\ell = 0$ for all $\ell$. (V is finite at the origin)  
 - $r>R$: $A_\ell = 0$ for all $\ell$.  (V --> 0 as r --> $\infty$
 - And the potential is continuous at $r=R$. 
 
This means that the solution for $r<R$ is 
$$ V(r, \theta) = \sum_\ell^\infty A_\ell r^\ell P_\ell(\cos\theta); $$

and for $r>R$, 
$$ V(r, \theta) = \sum_\ell^\infty \frac{B_\ell}{r^{\ell+1}} P_\ell(\cos\theta);$$

continutiy of the potential at $r=R$ implies 

$$ B_\ell = A_\ell R^{2\ell + 1} $$
for all $\ell$.  

At $r=R$, we also know that the potential equals $V_0(\theta)$.  This allows the coefficients $A_\ell$ (and, therefore, $B_\ell$) to be found by projecting onto the orthogonal Legendre polynomial basis functions: 

$$ A_\ell = \frac{2\ell + 1}{2R^\ell} \int_0^\pi V_0(\theta) P_\ell(\cos\theta) \sin\theta \, \mathrm{d}\theta.$$

We can then evaluate this integral in each of the cases below. The potential is azimuthally symmetric, so it's plotted in the y-z plane (x=0). The first example is $V_o(\theta)=sin\theta$. The second plot shows the $A_\ell$. Note that the potential resembles the potential for the 4 point charges found earlier. This is essentially a quadrupole + constant potential.


In [ ]:
# Define constants
R = 2.0

# Predefine a V_0 function
def V0(theta):
    return 2.0   #

In [ ]:
# Integration kernel for the A_\ell coefficients
def AlKernel(theta, ell):
    return V0(theta)*sf.eval_legendre(ell, np.cos(theta))*np.sin(theta)         

# Function to compute A_\ell coefficients
def AlCompute(L):            # L = number of terms in the sum
    Al = np.zeros(L)
    for ell in range(0, L):
        res = scipy.integrate.quad(AlKernel, 0.0, np.pi, args=(ell))
        Al[ell] = (2.0*ell + 1.0)*res[0]/(2.0*(R**ell))
    return Al
# Function to compute V(r, \theta)
def VCompute(r, theta, Al):
    L = len(Al)
    Vr = 0.0  # return value
    for ell in range(0, L):
        Vl = np.where(r < R, 
            Al[ell] * (r**ell) * sf.eval_legendre(ell, np.cos(theta)),
            Al[ell] * (R**(2*ell+1)) * sf.eval_legendre(ell, np.cos(theta))/(r**(ell+1)))
        Vr += Vl
    #return k*Vr
    return Vr

#Plot V(x,y)

def RunAndPlot(Al):
    NR = 100 
    colormap = cm.get_cmap('jet')
    rarr = np.linspace(0.001, 5.0*R, NR)
    thetaarr = np.linspace(0.0, 2*np.pi, NR)
    Rg, Theta = np.meshgrid(rarr, thetaarr)
    V = VCompute(Rg, Theta, Al)
    Z = Rg * np.cos(Theta)
    Y = Rg * np.sin(Theta)
    
    Vct = plt.contourf(Y, Z, V, 40, cmap=colormap)
    plt.plot(R*np.cos(thetaarr),R*np.sin(thetaarr), color='black')
    plt.contour(Y, Z, V, 20, colors='black')  #overplot line contours
    xx = plt.axes().set_aspect('equal')
    plt.colorbar(Vct)
    plt.xlabel("$y$")
    plt.ylabel("$z$")
    plt.xlim(-5.0, 5.0)
    plt.ylim(-5.0, 5.0)
    
    

In [ ]:
NR = 10 
rarr = np.linspace(0.001, 5.0*R, NR)
thetaarr = np.linspace(0.0, 2*np.pi, NR)
Rg, Theta = np.meshgrid(rarr, thetaarr)
Z = Rg * np.cos(Theta)
Y = Rg * np.sin(Theta)
#print(Rg[:,1])

In [ ]:
##### Define V_0, get A_ell, plot 
def V0(theta):
    return np.sin(theta)
L = 50
Al = AlCompute(L)
print(Al[0:10])   
RunAndPlot(Al)
plt.title(r"Potential for $V_o(\theta) = \sin(theta$)")



In [ ]:
# What terms contribute most to the expansion?

plt.bar(np.linspace(0,10,10),Al[0:10])
plt.title('Coefficients')
plt.show()

<div class="alert alert-block alert-warning">
Exercise 7: Plot $V(r,\theta)$ for the boundary conditions: a) $V(r,\theta) = cos\theta$, b)$V(r,\theta) = P_2(cos\theta)$, c) $V(r,\theta) = \theta + 0.5\theta^2$. Also plot the $A_\ell$. Comment on the results; in particular what is the electric field inside the sphere for case a)?  

In [ ]:
V0 = lambda theta : np.cos(theta)                                        #Exercise 7a solution
L = 50
Al = AlCompute(L)
print(Al[0:10])   
RunAndPlot(Al)
plt.title(r"Potential for $V_o(\theta) = \cos(\theta$)")
plt.figure()
plt.bar(np.linspace(0,10,10),Al[0:10])
plt.title('Coefficients')
plt.show()

In [ ]:
V0 = lambda theta : 0.5*(3*np.cos(theta)**2 - 1)                        #Exercise 7b solution
N = 50
Al = AlCompute(L)
print(Al[0:10])   
RunAndPlot(Al)
#plt.title(r"Potential due to sphere")
plt.title(r"Potential for $V_o(\theta) = P_2(\cos\theta$)")
plt.figure()
plt.bar(np.linspace(0,10,10),Al[0:10])
plt.title('Coefficients')
plt.show()

In [ ]:
V0 = lambda theta : theta + 0.5*theta**2
N = 50                                                                   #Exercise 7c solution
Al = AlCompute(L)
print(Al[0:10])   
RunAndPlot(Al)
#plt.title(r"Potential due to sphere")
plt.title(r"Potential for $V_o(\theta) = \theta + 0.5\theta^2 $")
plt.figure()
plt.bar(np.linspace(0,10,10),Al[0:10])
plt.title('Coefficients')
plt.show()

**Discussion 7:** Type your discussion here.





### 3.4 Expansion of a point charge in Legendre polynomials

If a point charge is not at the origin, the equipotentials will not be spherically symmetric about the origin.  The potential of a point charge $q$ on the z-axis at $\vec{r_o} = r_o\hat{z}$ can be expanded in Legendre polynomials:

$$V_q(r,\theta) = \frac{kq}{|\vec{r} - \vec{r}_o|} = \frac{kq}{(r^2+r_o^2 - 2rr_ocos\theta)^{1/2}}\qquad(1)$$


For $r<r_o$,  
$$ V_q(r, \theta) = kq                                                                                                                        \sum_{\ell=0}^\infty  \frac{r^\ell}{r_o^{\ell + 1}} P_\ell(\cos\theta)\qquad(2)$$ 

and for $r>r_o$, 
$$ V_q(r, \theta) = kq\sum_{\ell=0}^\infty  \frac{r_o^\ell}{r^{\ell + 1}} P_\ell(\cos\theta). \qquad  (3)$$

Remark: Eqs.(2) and (3) are called the "generating function" for the Legendre polynomials. For example, replacing $q\rightarrow \ \rho(r') d\tau'$,  $r_o\rightarrow r'$,and integrating over r' we have the solution to Poisson's equation

$$\nabla^2V(r,\theta)= \frac{\rho}{\epsilon_o}$$

If the point charge is outside any spherical surface of radius R ($R < r_o$) centered at the origin, then the potential on the sphere at r=R due to the point charge is
 
$$ V_q(\theta) = V_q(R, \theta) = kq \sum_{\ell=0}^\infty  \frac{R^\ell}{r_o^{\ell + 1}} P_\ell(\cos\theta)\qquad(4)$$ 

and if the charge is inside the sphere ($R > r_o$) then

$$ V_q(\theta) = V_q(R, \theta) = kq\sum_{\ell=0}^\infty  \frac{r_o^\ell}{R^{\ell + 1}} P_\ell(\cos\theta)\qquad(5).$$

### 3.5 Potential of a grounded conductor due to a nearby point charge - revisit the image charge solution

Eqs (4) and (5) above are the potentials on any surface of radius R centered on the origin due to a point charge, either inside or outside the sphere.  Now if we have a grounded conducting sphere, V=0 on the surface. Let's say we have a positive charge outside the sphere. What happens is that negative charge comes in from ground and the surface acquires a negative charge density arranged to exactly cancel the potential due to the positive point charge outside. It turns out that this surface charge density creates a potential outside the sphere that looks just like a negative point charge inside! We solved this problem before using the Method of images, which seemed to rely on a "guess" that really wasn't obvious, but it seemed to do the trick. We can do the same problem in a more systematic way by finding the potential on the sphere due to the exterior charge, then find the potential on the sphere that would be required to produce V=0 on the surface. Then we will see that the induced surface charge density is exactly what a point charge inside the sphere would produce. We can do all of this by expansions in Legendre polynomials.

This method is general and can be used to find the potential for any distribution of exterior charges.

**Notation** $V = V_{qa} + V_s$, where the total potential V is the sum of the potential due to the induced charge distribution on the sphere ($V_s$) and that due to the point charge q = qa at $r_o = a$. ($V_{qa}$). 



$$ V_s(r, \theta) = \sum_\ell^\infty A_\ell r^{\ell} P_\ell(\cos\theta).\ \  r  \le R \qquad(6)$$

$$ V_s(r, \theta) = \sum_\ell^\infty \frac{B_\ell}{r^{\ell+1}} P_\ell(\cos\theta).\ \  r \ge R \qquad(7)$$

The potential is continuous at r=R, so $B_\ell = A_\ell R^{2\ell+1}$.

Note that for a conductor, the total potential  $V = 0$ for r $\le R$.

**Restate the problem:** What is the total potential due to a point charge $q_a$ at $\vec{r_o} = a\hat{z}$? Where a > R, so the charge is outside a conducting sphere of radius R.

a)  First, ignore the fact that the sphere is a conductor for now, and just find the potential on the surface of the sphere due to the point charge qa outside. This potential is eq(4), call it $V_{qa}(R,\theta)$. 

b) Next, impose the condition that the sphere is conducting, so V = 0 at r = R. The potential due to the induced charge on the sphere  is found by solving $\nabla^2V_s = 0$, with $V_o(\theta) = - V_{qa}(R,\theta)$. This guarantees that V=0 at r=R.

We already have the boundary condition expressed in terms of a Legendre series expansion, so we don't have to project the BC onto the $P_{\ell}(cos\theta)$ to find the $A_{\ell}$ - we already have them from eqs.(4) and (5).

c) The total potential for r > R is found by adding: $V(r,\theta) = V_s(r,\theta) + V_{qa}(r,\theta)$. This  of course is the same as the potential found earlier using the method of images,because the solution is unique.  Also note that since V=0 on the surface of the conductor, we know that V=0 for $r \le R$ (and E=0 inside). 

d) Again, because we know the solution V_s is unique, we want to show that the Legendre polynomial expansion for the potential $V_s(R,\theta)$ on the sphere due to the exterior charge qa is the same as the potential due to the image charge qb inside, which is given by eq.(5). We know that the image charge is qb=- qa*R/a at $r_o = b = R^2/a$, so we'll find the potential on the surface due to this image charge. Of course the solution outside the sphere is uniquely determined by the boundary condition at r=R.

e) But wait! In part d) we are using the $\it{known}$ image charge. But what if we don't know that? The final step will be to show that the coefficients in the expansion of the potential due to the exterior charge qa allow us to find the image charge and its location within the sphere. 

Finally, since E = 0 inside the sphere, we can find the induced surface charge density from $\sigma(\theta) = - \epsilon_o \frac{\partial V_{out}}{\partial r}$ at r=R, where $V = V(r,\theta)$ for r > R. 

The default parameters for the charge outside the sphere and image charge are the 
same as those used for the image problem. 

<div class="alert alert-block alert-warning">
Exercise 8. Develop expressions for the $A_{\ell}$ for the potential of the sphere due to any exterior 
charge q at $r_o \ge R$ directly from the coefficients in eqs. (4) and (5). 
Show that $A_{\ell}=kq/r_o^{\ell+1}$ for a charge q at $r_o > R$, and 
$A_{\ell}=kqr_o^{\ell}/R^{2\ell+1}$ if $r_o < R$. 
We will now use these coefficients to compute $V_s(r,\theta)$.

###  Find the potential for the sphere due to the point charge q=1 at a=2R, both inside and outside the sphere.

In [ ]:
# Can easily find V_s using the coefficients from eq (4) and (5)  
qa = 1. #(q)
R = 2.
a = 2.*R  #position of point charge (r_o)
#a=.001
L = 100    # number of terms in the sum
Al = np.zeros(L)

# Use the $A_\ell$                      #These are the coefficients for V_s(r,theta)
for ell in range(0, L):
    if a > R:       
        Al[ell] = -  k*qa/(a**(ell + 1))     # qa outside sphere 
    else:
        Al[ell] = - k*qa*a**ell/(R**(2*ell + 1))   #qa inside sphere

print(Al[0:10])

RunAndPlot(Al) 
plt.plot(0,a,marker='o',color='black') # position of point charge
plt.title(r"  $(2) \qquad \qquad V_s(r,\theta) \qquad  V_{so}(\theta) = - V_{qa}(R,\theta)$")  

<div class="alert alert-block alert-warning">
Exercise 9. We took L=100 terms in the expansion. Do we really need that many? Plot the coefficients Al and comment on the result. 

**Discussion 9:** Type your discussion here.

In [ ]:
plt.bar(np.linspace(0,100,100),Al[0:100])               #Exercise 9 solution
plt.title('Coefficients')
plt.show()

<div class="alert alert-block alert-warning">
Exercise 10.  Edit the script to compute and plot the potential $V_s(r,\theta)$ due to the interior image charge qb=-qa(R/a) at b=$R^2/a$ and comment on the result. 

In [ ]:
q = 1.                                                          #Exercise 10 solution
R = 2.
qa=1
a = 2.*R 
b = R**2/a    # position of image charge
qb = - qa*R/a   #charge of image charge
#a=.001
L = 20    # number of terms in the sum         # reducing # of terms...not really necessary
Alimg = np.zeros(L)

# Use the $A_\ell$                      #These are the coefficients for V_s(r,theta)
for ell in range(0, L):
    if  b > R:       
        Alimg[ell] =   k*qb/(b**(ell + 1))     # if qb outside sphere 
    else:
        Alimg[ell] =  k*qb*b**ell/(R**(2*ell + 1))   # if qb inside sphere

print(Alimg[0:10])

RunAndPlot(Alimg) 
plt.plot(0,b,marker='o',color='white')            # plot position of image charge
plt.title(r"(3)     $ V_s(r,\theta),  V_{o}(\theta) =  V_{qb}(R,\theta)$")

**Discussion 10:** Type your discussion here.

<div class="alert alert-block alert-warning">
Exercise 11. Plot the $A_\ell$ for $ V_s(r,\theta)$, with  $V_{o}(\theta) =  V_{qb}(R,\theta)$ and $ V_s(r,\theta)$, with $V_{o}(\theta) =  V_{qa}(R,\theta)$.

In [ ]:
plt.bar(np.linspace(0,20,20),-Al[0:20])     #Exercise 11 solution
plt.bar(np.linspace(0,20,20),Alimg[0:20])
plt.title('Coefficients')
plt.show()

**Discussion 11:** Type your discussion here.

<div class="alert alert-block alert-warning">
Exercise 12: 

    Make sure you include enough terms in the expansion. Vary L and see what happens

- a) Plot the potential $V_s$ for a = R/2. (point charge inside the sphere).

- b) Plot $V_s$ for a close to the origin (maybe a=0.01 or less). 
- c) Repeat b), but leave out the L=0 term. This should show mainly the dipole contribution. 
- d) Find the induced surface charge density $\sigma(\theta)$ from the exterior charge qa and plot it.

In [ ]:
q = 1.                                                           #Exercise 12a solution
R = 22
a = R/2.  #position of point charge

qa = 1.

                                     

L = 20    # number of terms in the sum
Al = np.zeros(L)

for ell in range(0, L):
    if a > R:
        #Al[0] =  k*qa/a
        Al[ell] = k*qa/(a**(ell + 1))
    else:
        #Al[0] =  k*qa/R
        Al[ell] =  k*q*a**ell/(R**(2*ell+1))
        
print(Al[0:10])

# Compute V(r, \theta) and plot
RunAndPlot(Al)
plt.plot(0,a,marker='o',color='black') # position of point charge
plt.title(r"Point charge at $\vec{r_o} = a \hat{z}$"+'  a = '+str(a))

In [ ]:
R=2
a = 0.01  #position of point charge                          #Exercise 12b solution
qa=1
L = 50    # number of terms in the sum
Al = np.zeros(L)

for ell in range(0, L):
    if a > R:
        #Al[0] =  k*qa/a
        Al[ell] = k*qa/(a**(ell + 1))
    else:
        #Al[0] =  k*qa/R
        Al[ell] =  k*qa*(a**ell)/(R**(2*ell+1))
        
print(Al[0:10])

# Compute V(r, \theta) and plot
RunAndPlot(Al)
plt.plot(0,a,marker='o',color='black') # position of point charge
plt.title(r"Point charge at $\vec{r_o} = a \hat{z}$"+'  a = '+str(a))

In [ ]:
R=2
a = 0.01  #position of point charge                          #Exercise 12c solution
qa=1
L = 50    # number of terms in the sum
Al = np.zeros(L)

for ell in range(1, L):
    if a > R:
        #Al[0] =  k*qa/a
        Al[ell] = k*qa/(a**(ell + 1))
    else:
        #Al[0] =  k*qa/R
        Al[ell] =  k*qa*(a**ell)/(R**(2*ell+1))
        
print(Al[0:10])

# Compute V(r, \theta) and plot
RunAndPlot(Al)
plt.plot(0,a,marker='o',color='black') # position of point charge
plt.title(r"Point charge at $\vec{r_o} = a \hat{z}$"+'  a = '+str(a))

##### Let's reconstruct the full solution. Use the Legendre polynomial expansions to find $V(r,\theta)= V_s + V_{qa}$. 

DJ : Can you do 12c,d? I don't have time to do it. Shouldn't be that hard. 

**Discussion 12:** Type your discussion here.

<div class="alert alert-block alert-warning">
Exercise 13: Find the image charge and its location inside the sphere. Start with the Legendre series expansion
for $V_s(r,\theta)$ for $r\ge R$ (eq 7) and compare it to the potential of a point charge inside the sphere 
(eq 3). From this, you can find the image charge q=qb and and its location $r_o=b$. 
See the discussion in section 2 for the point charge outside a conducting sphere. 
    

**Discussion 13:** Type your discussion here.

### A way to plot a function of $\theta$ and $\phi$ on the sphere.
Plot spherical harmonics. **DJ: This needs some work, but I think I'll stop here. **

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import cm, colors     # a different way to get color tables
from mpl_toolkits.mplot3d import Axes3D
import scipy.special as sf     #special functions


# Plot a function of theta on the sphere
NR = 100
theta = np.linspace(0.0, 2*np.pi, NR)
phi = np.linspace(0, np.pi, 100)
phi, theta = np.meshgrid(phi, theta)
# can give some function of theta, e.g.
sigma = np.cos(theta)
sigplot = np.outer(np.ones(NR),sigma)  #make 2D array

# The Cartesian coordinates of the unit sphere
x = np.sin(phi) * np.cos(theta)
y = np.sin(phi) * np.sin(theta)
z = np.cos(phi)

#Calculate the spherical harmonic Y(l,m) and normalize to [0,1]

m, l = 0, 1    # for m = 0, Y(l,m) are basically the Legendre polynomials Pl(cos theta)
fcolors = sf.sph_harm(m, l, theta, phi).real
#fcolors = sigplot 
fmax, fmin = fcolors.max(), fcolors.min()
fcolors = (fcolors - fmin)/(fmax - fmin)

# Set the aspect ratio to 1 so the sphere looks spherical    -  except it doesn't!
fig = plt.figure(figsize=plt.figaspect(1.))
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(x, y, z,  rstride=1, cstride=1, facecolors=cm.jet(fcolors))
# Turn off the axis planes
ax.set_axis_off()
plt.show()
